In [1]:
import argparse
import copy
import os
import os.path as osp
import time
import warnings

import mmcv
import torch
from mmcv import Config, DictAction
from mmcv.runner import get_dist_info, init_dist
from mmcv.utils import get_git_hash

from mmdet import __version__
from mmdet.apis import set_random_seed, train_detector
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.utils import collect_env, get_root_logger


In [2]:
def parse_args():
    parser = argparse.ArgumentParser(description='Train a detector')
    parser.add_argument('config', default = "configs/svhn/mask_rcnn_swin_tiny_patch4_window7_mstrain_480-800_adamw_3x_svhn.py", help='train config file path')
    parser.add_argument('--work-dir',default = "output/", help='the dir to save logs and models')
    parser.add_argument(
        '--resume-from', help='the checkpoint file to resume from')
    parser.add_argument(
        '--no-validate',
        action='store_true',
        help='whether not to evaluate the checkpoint during training')
    group_gpus = parser.add_mutually_exclusive_group()
    group_gpus.add_argument(
        '--gpus',
        type=int,
        default = 1,
        help='number of gpus to use '
        '(only applicable to non-distributed training)')
    group_gpus.add_argument(
        '--gpu-ids',
        type=int,
        nargs='+',
        help='ids of gpus to use '
        '(only applicable to non-distributed training)')
    parser.add_argument('--seed', type=int, default=None, help='random seed')
    parser.add_argument(
        '--deterministic',
        action='store_true',
        help='whether to set deterministic options for CUDNN backend.')
    parser.add_argument(
        '--options',
        nargs='+',
        action=DictAction,
        help='override some settings in the used config, the key-value pair '
        'in xxx=yyy format will be merged into config file (deprecate), '
        'change to --cfg-options instead.')
    parser.add_argument(
        '--cfg-options',
        nargs='+',
        action=DictAction,
        help='override some settings in the used config, the key-value pair '
        'in xxx=yyy format will be merged into config file. If the value to '
        'be overwritten is a list, it should be like key="[a,b]" or key=a,b '
        'It also allows nested list/tuple values, e.g. key="[(a,b),(c,d)]" '
        'Note that the quotation marks are necessary and that no white space '
        'is allowed.')
    parser.add_argument(
        '--launcher',
        choices=['none', 'pytorch', 'slurm', 'mpi'],
        default='none',
        help='job launcher')
    parser.add_argument('--local_rank', type=int, default=0)
    args, unknown = parser.parse_known_args()
    if 'LOCAL_RANK' not in os.environ:
        os.environ['LOCAL_RANK'] = str(args.local_rank)

    if args.options and args.cfg_options:
        raise ValueError(
            '--options and --cfg-options cannot be both '
            'specified, --options is deprecated in favor of --cfg-options')
    if args.options:
        warnings.warn('--options is deprecated in favor of --cfg-options')
        args.cfg_options = args.options

    return args


In [3]:
args = parse_args()
print(args.config)
args.config = "configs/svhn/mask_rcnn_swin_tiny_patch4_window7_mstrain_480-800_adamw_3x_svhn.py"
#args.resume_from = "output/epoch_2.pth"

C:\Users\YaShu\AppData\Roaming\jupyter\runtime\kernel-db4dd90b-812a-48e9-b73e-0c5dddddd6d4.json


In [4]:
cfg = Config.fromfile(args.config)



if args.cfg_options is not None:
    cfg.merge_from_dict(args.cfg_options)
# import modules from string list.
if cfg.get('custom_imports', None):
    from mmcv.utils import import_modules_from_strings
    import_modules_from_strings(**cfg['custom_imports'])
# set cudnn_benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True
# work_dir is determined in this priority: CLI > segment in file > filename
if args.work_dir is not None:
    # update configs according to CLI args if args.work_dir is not None
    cfg.work_dir = args.work_dir
elif cfg.get('work_dir', None) is None:
    # use config filename as default work_dir if cfg.work_dir is None
    cfg.work_dir = osp.join('./work_dirs',
                            osp.splitext(osp.basename(args.config))[0])
if args.resume_from is not None:
    cfg.resume_from = args.resume_from
if args.gpu_ids is not None:
    cfg.gpu_ids = args.gpu_ids
else:
    cfg.gpu_ids = range(1) if args.gpus is None else range(args.gpus)
# init distributed env first, since logger depends on the dist info.
if args.launcher == 'none':
    distributed = False
else:
    distributed = True
    init_dist(args.launcher, **cfg.dist_params)
    # re-set gpu_ids with distributed training mode
    _, world_size = get_dist_info()
    cfg.gpu_ids = range(world_size)


In [5]:
# create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# dump config
cfg.dump(osp.join(cfg.work_dir, osp.basename(args.config)))
# init the logger before other steps
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
log_file = osp.join(cfg.work_dir, f'{timestamp}.log')
logger = get_root_logger(log_file=log_file, log_level=cfg.log_level)
# init the meta dict to record some important information such as
# environment info and seed, which will be logged
meta = dict()
# log env info
env_info_dict = collect_env()
env_info = '\n'.join([(f'{k}: {v}') for k, v in env_info_dict.items()])
dash_line = '-' * 60 + '\n'
logger.info('Environment info:\n' + dash_line + env_info + '\n' +
            dash_line)
meta['env_info'] = env_info
meta['config'] = cfg.pretty_text
# log some basic info
logger.info(f'Distributed training: {distributed}')
logger.info(f'Config:\n{cfg.pretty_text}')
# set random seeds
if args.seed is not None:
    logger.info(f'Set random seed to {args.seed}, '
                f'deterministic: {args.deterministic}')
    set_random_seed(args.seed, deterministic=args.deterministic)
cfg.seed = args.seed
meta['seed'] = args.seed
meta['exp_name'] = osp.basename(args.config)


2021-11-25 19:24:03,564 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: win32
Python: 3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]
CUDA available: True
GPU 0: NVIDIA GeForce RTX 2080
CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v10.2
NVCC: Cuda compilation tools, release 10.2, V10.2.89
GCC: n/a
PyTorch: 1.10.0
PyTorch compiling details: PyTorch built with:
  - C++ Version: 199711
  - MSVC 192829337
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 10.2
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;ar

In [6]:
model = build_detector(
    cfg.model,
    train_cfg=cfg.get('train_cfg'),
    test_cfg=cfg.get('test_cfg'))
model.init_weights()
datasets = [build_dataset(cfg.data.train)]
if len(cfg.workflow) == 2:
    val_dataset = copy.deepcopy(cfg.data.val)
    val_dataset.pipeline = cfg.data.train.pipeline
    datasets.append(build_dataset(val_dataset))
if cfg.checkpoint_config is not None:
    # save mmdet version, config file content and class names in
    # checkpoints as meta data
    cfg.checkpoint_config.meta = dict(
        mmdet_version=__version__ + get_git_hash()[:7],
        CLASSES=datasets[0].CLASSES)
# add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES


C:\Users\YaShu\Anaconda3\envs\openmmlab\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
C:\Users\YaShu\Desktop\google drive(nctu)\python\CBNetV2-main\CBNetV2-main\mmdet\core\anchor\builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '
2021-11-25 19:24:09,049 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2021-11-25 19:24:09,066 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2021-11-25 19:24:09,071 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override':

loading annotations into memory...
Done (t=0.57s)
creating index...
index created!


In [7]:
print(model)

MaskRCNN(
  (backbone): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0): BasicLayer(
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=96, out_features=288, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=96, out_features=96, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=96, out_features=384, bias=Tru

In [8]:
for p in model.parameters():
    p.requires_grad = False
for name, p in model.named_parameters():
    if "roi_head.bbox_head" in name:
        p.requires_grad = True
    if "roi_head.mask_head" in name:
        p.requires_grad = True
    if "neck" in name:
        print(name)
        p.requires_grad = True

neck.lateral_convs.0.conv.weight
neck.lateral_convs.0.conv.bias
neck.lateral_convs.1.conv.weight
neck.lateral_convs.1.conv.bias
neck.lateral_convs.2.conv.weight
neck.lateral_convs.2.conv.bias
neck.lateral_convs.3.conv.weight
neck.lateral_convs.3.conv.bias
neck.fpn_convs.0.conv.weight
neck.fpn_convs.0.conv.bias
neck.fpn_convs.1.conv.weight
neck.fpn_convs.1.conv.bias
neck.fpn_convs.2.conv.weight
neck.fpn_convs.2.conv.bias
neck.fpn_convs.3.conv.weight
neck.fpn_convs.3.conv.bias


In [ ]:
train_detector(
    model,
    datasets,
    cfg,
    distributed=distributed,
    validate=(not args.no_validate),
    timestamp=timestamp,
    meta=meta)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
loading annotations into memory...


2021-11-25 19:24:10,543 - mmdet - INFO - load checkpoint from local path: checkpoints/latest.pth


Done (t=0.21s)
creating index...
index created!


2021-11-25 19:24:10,832 - mmdet - INFO - Start running, host: YaShu@DESKTOP-7QM91N0, work_dir: C:\Users\YaShu\Desktop\google drive(nctu)\python\CBNetV2-main\CBNetV2-main\output
2021-11-25 19:24:10,832 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(ABOVE_NORMAL) DistOptimizerHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(LOW         ) IterTimerHo

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


...

2021-11-25 19:26:01,001 - mmdet - INFO - Epoch [1][650/20730]	lr: 1.000e-04, eta: 3:52:22, time: 0.154, data_time: 0.003, memory: 1128, loss_rpn_cls: 0.0029, loss_rpn_bbox: 0.0092, loss_cls: 0.1007, acc: 96.7930, loss_bbox: 0.1514, loss_mask: 0.2246, loss: 0.4888
2021-11-25 19:26:08,250 - mmdet - INFO - Epoch [1][700/20730]	lr: 1.000e-04, eta: 3:49:49, time: 0.145, data_time: 0.003, memory: 1128, loss_rpn_cls: 0.0014, loss_rpn_bbox: 0.0081, loss_cls: 0.0931, acc: 96.6172, loss_bbox: 0.1525, loss_mask: 0.2095, loss: 0.4647
2021-11-25 19:26:15,833 - mmdet - INFO - Epoch [1][750/20730]	lr: 1.000e-04, eta: 3:48:13, time: 0.152, data_time: 0.003, memory: 1128, loss_rpn_cls: 0.0016, loss_rpn_bbox: 0.0114, loss_cls: 0.0845, acc: 96.7852, loss_bbox: 0.1439, loss_mask: 0.2343, loss: 0.4755
2021-11-25 19:26:23,265 - mmdet - INFO - Epoch [1][800/20730]	lr: 1.000e-04, eta: 3:46:32, time: 0.149, data_time: 0.003, memory: 1128, loss_rpn_cls: 0.0016, loss_rpn_bbox: 0.0100, loss_cls: 0.0832, acc: 96.6

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0


2021-11-25 19:26:38,163 - mmdet - INFO - Epoch [1][900/20730]	lr: 1.000e-04, eta: 3:43:45, time: 0.149, data_time: 0.003, memory: 1128, loss_rpn_cls: 0.0023, loss_rpn_bbox: 0.0089, loss_cls: 0.0865, acc: 96.9766, loss_bbox: 0.1379, loss_mask: 0.8322, loss: 1.0677
2021-11-25 19:26:45,932 - mmdet - INFO - Epoch [1][950/20730]	lr: 1.000e-04, eta: 3:43:01, time: 0.155, data_time: 0.003, memory: 1128, loss_rpn_cls: 0.0045, loss_rpn_bbox: 0.0100, loss_cls: 0.1020, acc: 96.1758, loss_bbox: 0.1745, loss_mask: 0.2256, loss: 0.5167
2021-11-25 19:26:53,257 - mmdet - INFO - Exp name: mask_rcnn_swin_tiny_patch4_window7_mstrain_480-800_adamw_3x_svhn.py
2021-11-25 19:26:53,257 - mmdet - INFO - Epoch [1][1000/20730]	lr: 1.000e-04, eta: 3:41:44, time: 0.146, data_time: 0.003, memory: 1128, loss_rpn_cls: 0.0009, loss_rpn_bbox: 0.0083, loss_cls: 0.0829, acc: 96.8750, loss_bbox: 0.1450, loss_mask: 0.2144, loss: 0.4513
2021-11-25 19:27:00,555 - mmdet - INFO - Epoch [1][1050/20730]	lr: 1.000e-04, eta: 3:40:

2021-11-25 19:30:29,755 - mmdet - INFO - Epoch [1][2450/20730]	lr: 1.000e-04, eta: 3:27:25, time: 0.149, data_time: 0.003, memory: 1128, loss_rpn_cls: 0.0010, loss_rpn_bbox: 0.0092, loss_cls: 0.0905, acc: 96.6445, loss_bbox: 0.1597, loss_mask: 0.2116, loss: 0.4720
2021-11-25 19:30:37,458 - mmdet - INFO - Epoch [1][2500/20730]	lr: 1.000e-04, eta: 3:27:16, time: 0.154, data_time: 0.003, memory: 1128, loss_rpn_cls: 0.0010, loss_rpn_bbox: 0.0072, loss_cls: 0.0749, acc: 97.1172, loss_bbox: 0.1346, loss_mask: 0.1975, loss: 0.4153
2021-11-25 19:30:44,970 - mmdet - INFO - Epoch [1][2550/20730]	lr: 1.000e-04, eta: 3:27:01, time: 0.150, data_time: 0.003, memory: 1128, loss_rpn_cls: 0.0012, loss_rpn_bbox: 0.0103, loss_cls: 0.0662, acc: 97.3555, loss_bbox: 0.1354, loss_mask: 0.2267, loss: 0.4399
2021-11-25 19:30:52,249 - mmdet - INFO - Epoch [1][2600/20730]	lr: 1.000e-04, eta: 3:26:39, time: 0.146, data_time: 0.003, memory: 1128, loss_rpn_cls: 0.0013, loss_rpn_bbox: 0.0080, loss_cls: 0.0748, acc: 

2021-11-25 19:34:19,645 - mmdet - INFO - Exp name: mask_rcnn_swin_tiny_patch4_window7_mstrain_480-800_adamw_3x_svhn.py
2021-11-25 19:34:19,645 - mmdet - INFO - Epoch [1][4000/20730]	lr: 1.000e-04, eta: 3:20:11, time: 0.149, data_time: 0.003, memory: 1128, loss_rpn_cls: 0.0025, loss_rpn_bbox: 0.0088, loss_cls: 0.0717, acc: 97.0234, loss_bbox: 0.1503, loss_mask: 0.2079, loss: 0.4411
2021-11-25 19:34:27,221 - mmdet - INFO - Epoch [1][4050/20730]	lr: 1.000e-04, eta: 3:20:03, time: 0.152, data_time: 0.003, memory: 1128, loss_rpn_cls: 0.0015, loss_rpn_bbox: 0.0098, loss_cls: 0.0693, acc: 97.2734, loss_bbox: 0.1437, loss_mask: 0.2112, loss: 0.4356
2021-11-25 19:34:34,675 - mmdet - INFO - Epoch [1][4100/20730]	lr: 1.000e-04, eta: 3:19:52, time: 0.149, data_time: 0.003, memory: 1128, loss_rpn_cls: 0.0015, loss_rpn_bbox: 0.0111, loss_cls: 0.0799, acc: 96.7891, loss_bbox: 0.1627, loss_mask: 0.2168, loss: 0.4719
2021-11-25 19:34:42,231 - mmdet - INFO - Epoch [1][4150/20730]	lr: 1.000e-04, eta: 3:1